In [18]:
# load packages
import numpy as np 
from scipy.io   import  loadmat
from scipy import stats
import pandas as pd
import os
import matplotlib.pyplot as plt #import matplotlib as plt
from scipy.optimize import curve_fit 
import seaborn as sns #import mat73
import pickle as pkl
from datetime import datetime
import statsmodels.api as sm 
from statsmodels.formula.api import ols

In [19]:
# #perform two-way ANOVA
# model = ols('height ~ C(water) + C(sun) + C(water):C(sun)', data=df).fit()
# sm.stats.anova_lm(model, typ=2)

#### import and set actual anova params

In [20]:
# Main Directory of processed file from MatLab
#MainDir = 'D:\\AttnXV3_analysis\\RCA_F1\\AvgCRFs\\' # set dir
MainDir = 'C:\\plimon\\LTP_analysis\\RCA_F1\\AmpResponseDiffs\\' # set dir
os.chdir(MainDir) # change old dir, to this dir
d = os.listdir(MainDir) # list files in dir
print(f'Files on hand: {d}')
##############################################
FileN = d[0] # choose one                        
file_path1 = os.path.join(MainDir, FileN) # join paths and prep 2 load
print('Current WD:',file_path1) # does path exist ... ?
print('Does File #1 Exist?',os.path.exists(file_path1)) # yes or no

Files on hand: ['LTP_ResponseDifferences_20240223_1325.pkl', 'PostPreResponseDifferences_20240220_1029.pkl']
Current WD: C:\plimon\LTP_analysis\RCA_F1\AmpResponseDiffs\LTP_ResponseDifferences_20240223_1325.pkl
Does File #1 Exist? True


In [21]:
loadData = pkl.load(open(file_path1,'rb'))
print(loadData.keys())

dict_keys(['DiffData', 'ErrVals', 'SubNames', 'DictMainKeys', 'ContLevs', 'crfLabs', 'VoiceMemo'])


In [22]:
data = loadData['DiffData']
# wont load error bars just yet
data_labels = loadData['DictMainKeys']
SubNames = np.array(loadData['SubNames'])
note = loadData['VoiceMemo']
contrast_levels = loadData['ContLevs']
contrast_labs = loadData['crfLabs']
print(note)
NumSubs = int(len(SubNames))
print(data_labels) # labels for nr plots

['Cont:Post - pre responses for all conditions for all subs, sub x contrast diff']
['attnL F1', 'attnL F2', 'attnR F1', 'attnR F2']


#### Stack all condition differences for 1 subject - make into a function

In [23]:
NumBins = 6 # number of contrasts
NumHarms = 2 # number of harmonic data: 2F1, 4F1
NumComp = 0 # first component from RCA
NumConds = 4

In [24]:
RowPop = NumBins * NumHarms * NumConds # 6 x 2 x 4
SubRows = NumBins*NumHarms
ContrastInd = np.arange(0,NumBins)
ContrastArr = np.arange(RowPop) % NumBins

In [25]:
AttnXArr = np.zeros((RowPop))
FreqFiltArr = np.zeros((RowPop))
HarmArr = np.zeros((RowPop))
# load subject response differnces across conditions
for co in range(NumConds):
    HarmArr[(co*SubRows):(co+1)*SubRows]  = [0] * NumBins + [1] * NumBins # index harmonic data:  2F / 4F 
if co == 0 or co == 1: # Index if Data is AttnL or AttnR
    AttnXArr[(co*SubRows):(co+1)*(SubRows)] = 0
else:
    AttnXArr[(co*SubRows):(co+1)*(SubRows)] = 1
if co % 2:          # Index what data was F1 = 6 Hz or F2 = 7.5 Hz 
    FreqFiltArr[(co*SubRows):(co+1)*SubRows] = [1]*SubRows
else:
    FreqFiltArr[(co*SubRows):(co+1)*SubRows] = [0]*SubRows

In [11]:
def MakeDataFramePerSubject(RowPop,SubRows,ContrastArr,AttnXArr,FreqFiltArr,HarmArr,NumConds,soi, dictIn):
    SubjArr = [soi] * RowPop # Subject label
    ValArr = np.zeros((RowPop)) # INDEPENDENT VARIABLE 12 x 4  = 48 rows per val
    # load subject response differnces across conditions
    for co in range(NumConds):
        dIn = dictIn[co][soi,:] 
        ValArr[(co*SubRows):(co+1)*SubRows] = dIn # Sort Data Values 
    
    dataOut = {'Subj': SubjArr,
                'AttnX': AttnXArr,
                'FreqFilt': FreqFiltArr,
                'Harmonic': HarmArr,
                'Contrast': ContrastArr,
                'Vals': ValArr}
    df = pd.DataFrame(dataOut)
    return df

#### Load Some DateFrames to see if they differ

In [27]:
#print(Sub_df[22])

In [30]:
Sub_df = {}
for su in range(NumSubs):
    Sub_df[su]= MakeDataFramePerSubject(RowPop,SubRows,ContrastArr,AttnXArr,FreqFiltArr,HarmArr,NumConds,soi=su,dictIn=data)

In [39]:
anova_array = np.zeros((RowPop*NumSubs,6))
print(anova_array.shape)

headerArr =list(Sub_df[0].columns.values)
print(headerArr)

(1200, 6)
['Subj', 'AttnX', 'FreqFilt', 'Harmonic', 'Contrast', 'Vals']


In [40]:
for su in range(NumSubs):
    aIn = np.array(Sub_df[su])
    anova_array[(RowPop*su):(RowPop*(su+1)),:] = aIn

In [45]:
anova_df = pd.DataFrame(anova_array, columns = headerArr)
print(anova_df)

      Subj  AttnX  FreqFilt  Harmonic  Contrast      Vals
0      0.0    0.0       0.0       0.0       0.0 -0.053774
1      0.0    0.0       0.0       0.0       1.0 -0.395514
2      0.0    0.0       0.0       0.0       2.0  0.293846
3      0.0    0.0       0.0       0.0       3.0  0.278024
4      0.0    0.0       0.0       0.0       4.0 -0.058382
...    ...    ...       ...       ...       ...       ...
1195  24.0    1.0       1.0       1.0       1.0 -0.210037
1196  24.0    1.0       1.0       1.0       2.0  0.035446
1197  24.0    1.0       1.0       1.0       3.0  0.010592
1198  24.0    1.0       1.0       1.0       4.0  0.058085
1199  24.0    1.0       1.0       1.0       5.0 -0.155763

[1200 rows x 6 columns]


#### Run sum anova

In [49]:
#perform two-way ANOVA
model = ols('Vals ~ C(Harmonic) + C(Contrast) + C(AttnX)', data=anova_df).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(Harmonic),0.036925,1.0,0.076461,0.782200
C(Contrast),2.016581,5.0,0.835161,0.524682
C(AttnX),2.355269,1.0,4.877137,0.027404
Residual,575.641093,1192.0,NaN,NaN
